## Set up parameters

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [ ]:
img_width, img_height = 150,150

train_data_dir = '../data/train'
validation_data_dir = '../data/valid'
test_data_dir = '../data/test'

epochs = 50
batch_size = 32
from utils import *

In [ ]:
if K.image_dim_ordering() == 'th':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [ ]:
input_shape 

## Define and train model

In [ ]:
# dropout
p = 0.5

In [ ]:
model = Sequential()
model.add(Conv2D(32, 3, 3, input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(p))

model.add(Dense(3))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
# new for this version: adam instead of rmsprop
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
from PIL import Image
Image.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# augmentation configuration for training

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


# augmentation configuration we will use for validation and testing:
# only rescaling

validation_datagen = ImageDataGenerator(rescale=1. / 255)


validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
# code snippet to save weights every time the validation loss decreases
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath="../models/23-apr.h5", 
                               verbose=1, 
                               save_best_only=True, 
                               save_weights_only=True)

In [ ]:
# make sure samples_per_epoch is divisible by batch_size
history = model.fit_generator(train_generator,
                              samples_per_epoch=train_generator.n,
                              nb_epoch=epochs,
                              validation_data=validation_generator,
                              nb_val_samples=validation_generator.n)

In [ ]:
# visualize loss improvements over the run
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)
# "bo" is for "blue dot"
plt.plot(epochs, loss_values, 'bo')
# b+ is for "blue crosses"
plt.plot(epochs, val_loss_values, 'b+')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
# visualize accuracy improvements over the run
plt.clf()
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']
plt.plot(epochs, acc_values, 'bo')
plt.plot(epochs, val_acc_values, 'b+')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

## Score test set and prepare file for submission

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle = False,
    class_mode=None)

In [ ]:
preds = model.predict_generator(test_generator, val_samples = test_generator.n)

In [ ]:
preds.shape

In [ ]:
preds[:15]

In [ ]:
preds = preds.clip(min=0.05, max=0.95)

In [ ]:
#Extract imageIds from the filenames in our test/unknown directory 
filenames = test_generator.filenames
import numpy as np
ids = np.array([int(f[8:f.find('.')]) for f in filenames])

In [ ]:
filenames[:5]

In [ ]:
ids = ids.reshape(512,1)

In [ ]:
subm = np.concatenate((ids,preds),axis=1)

In [ ]:
subm[:5]

In [ ]:
submission_file_name = '23-april-submission1.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f,%.5f,%.5f', header='image_name,Type_1,Type_2,Type_3', comments='')